In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packa

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-06 21:16:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.71MB/s    in 0.2s    

2022-06-06 21:16:51 (5.71 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|                Y|
|R3LGC3EKEG84PX|          5|            0|          0|   N|                Y|
| R9PYL3OYH55QY|          5|            0|          1|   N|                Y|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|                Y|
|R15LYP3O51UU9E|          5|            0|          0|   N|                Y|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|                Y|
|R32FE8Y45QV434|          5|            0|          0|   N|                Y|
|R3NM4MZ4XWL43Q|          5|            1|          2|   N|                Y|
|R3H4FXX6Q7I37D|          4|            0|          0|   N|                Y|
|R30L5PET7LFFDC|          5|            1|          1|   N|     

In [6]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for columns with total votes is greater than 20
vine_greater20votes_df = vine_df.filter(col("total_votes") >= 20)
vine_greater20votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|
| RFN4PNRUD1UW2|          4|           21|         22|   N|                N|
| RO7EBJEP7IHIX|          5|           26|         26|   N|                N|
|R1CVS4MK9RTNNP|          2|           11|         22|   N|     

In [9]:
# Create a new df with column of ratio of helpful vs total votes
vine_helpful_ratio_df = vine_greater20votes_df.withColumn("helpful_ratio", vine_greater20votes_df["helpful_votes"] / vine_greater20votes_df["total_votes"])
vine_helpful_ratio_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|      helpful_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y| 0.9615384615384616|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N| 0.9615384615384616|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|               0.95|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y| 0.9047619047619048|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N| 0.9583333333333334|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|                1.0|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N| 0.9733333333333334|
| RFN4PNRUD1UW2|    

In [11]:
# Filter for all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
vine_helpful_votes_df = vine_helpful_ratio_df.filter(col("helpful_ratio") >= .5 )
vine_helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     helpful_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|0.9615384615384616|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|0.9615384615384616|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|              0.95|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|0.9047619047619048|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|0.9583333333333334|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|               1.0|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|0.9733333333333334|
| RFN4PNRUD1UW2|          4|  

In [15]:
vine_helpful_votes_df.count()

105986

In [13]:
# all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
vine_paid_df = vine_helpful_votes_df.filter(col("vine") == "Y")
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     helpful_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R84VUCDBCI29U|          3|           18|         28|   Y|                N|0.6428571428571429|
| R4V3ICFDTIDIF|          4|           20|         21|   Y|                N|0.9523809523809523|
|R1JZ0JAPW83WFS|          4|           54|         58|   Y|                N|0.9310344827586207|
|R1XH7EA97FAVP7|          3|           35|         44|   Y|                N|0.7954545454545454|
|R1482JAU1ZR7QH|          4|           15|         22|   Y|                N|0.6818181818181818|
|R1GGYGVTHP84DG|          4|           25|         27|   Y|                N|0.9259259259259259|
| RXGU9DSKZJSP0|          3|           21|         22|   Y|                N|0.9545454545454546|
+--------------+-----------+--

In [14]:
# all the rows where a review was written as part of the Vine program (unpaid), vine == 'N'
vine_unpaid_df = vine_helpful_votes_df.filter(col("vine") == "N")
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     helpful_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2SHXRL6SL1GC9|          3|           25|         26|   N|                Y|0.9615384615384616|
|R2ZC033X86YOY8|          5|           25|         26|   N|                N|0.9615384615384616|
|R2736RJGCOSL80|          5|           19|         20|   N|                Y|              0.95|
| RRY5DJ6J9BKAX|          5|           19|         21|   N|                Y|0.9047619047619048|
|R2P4PJJ2ROTPBM|          5|           46|         48|   N|                N|0.9583333333333334|
| RO8RAEGB66BKR|          4|           46|         46|   N|                N|               1.0|
| RRFZ7QZTRJC59|          5|          292|        300|   N|                N|0.9733333333333334|
| RFN4PNRUD1UW2|          4|  

In [20]:
# Total number of paid reviews
total_paid_votes = vine_paid_df.count()
print(total_paid_votes)

7


In [21]:
# Total number of unpaid reviews
total_unpaid_votes = vine_unpaid_df.count()
print(total_unpaid_votes)

105979


In [22]:
# Total number of 5 star reviews of paid reviews
five_star_paid_votes = vine_paid_df.filter(col("star_rating") == 5).count()
print(five_star_paid_votes)

0


In [23]:
# Total number of 5 star reviews of unpaid reviews
five_star_unpaid_votes = vine_unpaid_df.filter(col("star_rating") == 5).count()
print(five_star_unpaid_votes)

67580


In [27]:
# Percentage of 5 star reviews for paid reviews
paid_percent_five_star = (five_star_paid_votes / total_paid_votes)*100
print(paid_percent_five_star)

0.0


In [31]:
# Percentage of 5 star reviews for unpaid reviews
unpaid_percent_five_star = (five_star_unpaid_votes / total_unpaid_votes)*100
print(unpaid_percent_five_star)

63.767350135404186
